In [ ]:
# this is the main class
# facilitating property extraction and plotting
from MDPropTrack.analysis import PropertyAnalyser

# these are additional classes 
# implementing calculation of some key properties for lipids and proteins
from MDPropTrack.analysis import LipidPropertyCalculator, ProteinPropertyCalculator

`MDPropTrack` builds on [`MDAnalysis`](https://www.mdanalysis.org/) library for MD trajectory analysis.It's highly recommended to familiarise yourself with the Universe class structure, trajectory processing and transformation, and selection language.

`PropertyAnalyser` is the main class of the package. Apart from property extraction, it provides flexible plotting of time series.

You can also assess convergence of the properties by ploting autocorrelation time vs simulation time.
An indicator of time series convergence is that autocorrelation time plot reaches a plateau.

The methods for autocorrelation analysis in `PropertyAnalyser` were adapted from [emcee tutorial](https://emcee.readthedocs.io/en/stable/tutorials/autocorr/). I also suggest reading the following paper on [convergence in Molecular Dynamics](https://www.nature.com/articles/s42004-024-01114-5#Sec2) to get the idea of autocorrelation applicability.

**Let's overview what you can do with `PropertyAnalyser`**

Quick start:

In [ ]:
# create analyser for your system and get properties from an edr file
pa = PropertyAnalyser(edr='path/to/simulation.edr')

# call the following method to extract properties into DataFrame
pa.extract_properties()

# look at the DataFrame
pa.data

# plot extracted properties as time series
pa.plot()

# or plot autocorrelation to assess convergence
pa.plot(plot_convergence=True)

Now, let's get into details:

In [ ]:
# here are all the options for PropertyAnalyser

pa = PropertyAnalyser(

    # you can read properties from multiple edr files and/or trajectories
    edr = ['path/to/equilibration.edr', 'path/to/production.edr'],

    # you can provide edr without trj and vice versa
    # trajectories and topologies can be in any format compatible with MDAnalysis
    # note that trajectories and edrs are matched by file name
    # e.g. equilibration.edr and equilibration.xtc are considered to desribe the same simulation
    # the number of trajectories and edrs doesn't have to be the same
    trj = ['path/to/equilibration.xtc', 'path/to/production.xtc'],

    # for trajectories it's highly recommended to provide a topology file
    # tpr is the best option for GROMACS trajectories
    topol = 'path/to/equilibration.tpr',

    # NOTE! there is no default set of properties to calculate from trajectories
    # you need to provide a list of functions
    # here are ready-to-use options implemented in the package 
    funcs = [

        # LIPID PROPERTIES
        # see LiPyPhilic package documentation for more

        # lipid_sel - atom selection for lipids in the bilayer
		# These atoms will also be used to perform the Voronoi tessellation.
		# leaflet - leaflets to use for property averaging
        # -1, 1 or 0 (lower, upper and both respectively), default 0
        LipidPropertyCalculator(lipid_sel='name PO4').CalcAreaPerLipid,
        

        # lipid_sel - atoms to use for bilayer identification and thickness calculation
        LipidPropertyCalculator(lipid_sel='name PO4').CalcBilayerThickness,
        
        # average orientational order parameter over one or 2 tails
        # tail_sel - atom selection(s) for lipid tails
        # example below is two selections for sn1 an sn2 tails in MARTINI bead naming
        LipidPropertyCalculator(tail_sel=["name ??A", "name ??B"]).CalcOrderParameter,
        
        # NOTE!
        # instead, calculation of these properties can be combined in one go as
        LipidPropertyCalculator(lipid_sel='name PO4', tail_sel=["name ??A", "name ??B"], calculate=['apl', 'thickness', 'order_param']).CalcProps,
        
        # PROTEIN PROPERTIES

        # Gyration radius
        # protein_sel - one or multiple atom selections for Rg calculation
        ProteinPropertyCalculator(protein_sel=['chain A and resid 10:40', 'chain B']).CalcGyrationRadius

        # RMSD 
        # protein_sel - one or multiple atom selections for RMSD calculation
        # fit_sel - atom selection for superimposition
        ProteinPropertyCalculator(protein_sel=['chain A and backbone', 'chain B and backbone'], fit_sel='backbone').CalcRMSD
    ],

    # you can pass a list of names for properties calculated by each function
    # otherwise they will be named Prop1, Prop2 etc.
    func_names = ['AreaPerLipid', 'BilayerThickness', 'OrderParameter', 'Rg_A', 'Rg_B', 'RMSD_chain_A', 'RMSD_chain_B'],

    # by default trajectories are transformed by uwrapping atom groups in a simulation box
    # you can also specify groups for centering and rot-trans fit, default 'protein' 
    center_group='protein',
    rot_trans_group='protein' 
)

In [ ]:
# you can also tune property extraction

pa.extract_properties(
    
    # time in edrs and trajectories is usually reported in ps
    # and by default PropertyAnalyser converts it to ns
    # pass 'ps' to stay with ps
    tu='ps',

    # a step for trajectory analysis, default 1
    # only relevant for trajectories
    step=1,
    
    # if you pass a list of edrs/trajectories they will be treated as sequential steps
    # but maybe you did several equilibrations with different conditions and want to compare them
    # pass False to treat input steps as independent runs starting at 0 ps
    sequential=False,
    
    # reading edr is very fast so verbose will only report progress for trajectory analysis
    # default False
    verbose=True
)

**Note:** trajectory transformations can take 10s of seconds to minutes, so you might have to wait when running multiple analyses on big systems. 

`pa.data` is a pandas DataFrame so you can manipulate it as such (but keep in mind that plotting functions rely on a certain df structure).

For example:

In [ ]:
# see a list of properties in the DataFrame
print(pa.data.columns)

# save data as csv
pa.data.to_csv('path/to/file.csv')

# reload data into new class instance
pa_new = PropertyAnalyser()
pa_new.data = pd.read_csv('path/to/file.csv', index_col=0)

There are many ways to tune plotting too:

In [ ]:
pa.plot(

    # this is a default list of properties for plotting
    # pass any list of colum names from pa.data
    properties_to_plot=['Potential', 'Temperature', 'Pressure', 'Volume'],
    
    # if True, autocorrelation time plots will be created
    # instead of time series  
    plot_convergence=True,

    # a list of cudtom labels to use for legend
    labels=['Equilibration', 'Production'],

    # custom label for x axis
    x_lab='Time, ns',

    # seaborn color map to use
    cmap='Set1',

    # key word argumernts to manupulate creation of the figure
    figure_kwargs=None, 

    # seaborn style kwargs, here are the default options
    style_kwargs={"style": "darkgrid", "rc": {"grid.color": ".6", "grid.linestyle": ":"}},
    
    # seaborn plotting kwargs
    sns_kwargs={'alpha': 0.7}
)

`pa.plot()` returns figure and axes, which you can manipulate further *(just like `fig, axs = plt.subplots()`)*. 

**To calculate more properties from trajectories**, you can create custom functions.

Here is a template compatible with `PropertyAnalyser`. You can also look into MDPropTrack code for examples.

Note that one function can return an array of just one or multiple properties, so that return can be either `np.array((n_frames,))` or `np.array((n_frames, n_properties))` respectively.

In [ ]:
class CustomPropertyCalculator:
	"""
	Template for a class with methods to calculate
	properties from a trajectory
	"""

	def __init__(self, param):
		"""
		Class atributes hold parameters to be used in methods
		"""
		self.param = param

	def CalcOneProperty(self, system, step=1, verbose=False):
		"""
		Calculate a property

        # THE FOLLOWING ARGUMENTS ARE REQUIRED
        # DON'T ADD OTHER ARGUMENTS, THEY SHOULD BE PASSED AS CLASS ATTRIBUTES self.param
        system - MDAnalysis Universe, trajectory for analysis
        step - int, step for trajectory analysis
        verbose - bool, report trajectory analysis progress
        
        Returns
        np.array(floats)
        """
        
        # iterate over trj and calculate property
        property_vals = []
        iterator = system.trajectory[::step]	
        for ts in (tqdm(iterator) if verbose else iterator):
            
            # do some steps to calculate your desired property
            # taking into accound self.param
            val = ...
            
            property_vals.append(val)
            
        return np.array(property_vals)

    def CalcTwoProperties(self, system, step=1, verbose=False):
        """
        Calculate two properties in one go
        
        # THE FOLLOWING ARGUMENTS ARE REQUIRED
        # DON'T ADD OTHER ARGUMENTS, THEY SHOULD BE PASSED AS CLASS ATTRIBUTES self.param
        system - MDAnalysis Universe, trajectory for analysis
        step - int, step for trajectory analysis
        verbose - bool, report trajectory analysis progress
        
        Returns
        np.array(floats)
        """
        
        # iterate over trj and calculate property
        property_vals = []
        iterator = system.trajectory[::step]	
        for ts in (tqdm(iterator) if verbose else iterator):
            
            # do some steps to calculate your desired property
            # taking into accound self.param
            val1 = ...
            val2 = ...
            
            property_vals.append([val1, val2])
            
        return np.array(property_vals)